In [1]:
!conda activate llm

In [2]:
pip install torch==2.1.1 torchvision==0.16.1 torchaudio==2.1.1 --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print (device)

cuda


In [9]:
pip install tbb -q

Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install spacy -q

Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for spacy from https://files.pythonhosted.org/packages/39/e1/08681583569f435347ced0535b27c073fcc9a927d9b4293c963092f2d01c/spacy-3.7.5-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for spacy-legacy<3.1.0,>=3.0.11 from https://files.pythonhosted.org/packages/c3/55/12e842c70ff8828e34e543a2c7176dac4da006ca6901c9e8b43efab8bc6b/spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata
  Obtaining dependency information for spacy-loggers<2.0.0,>=1.0.0 from https://files.pythonhosted.org/packages/33/78/d1a1a026ef3af911159398c939b1509d5c36fe524c7b644f34a5146c4e16/spacy_loggers-1.0.5-py3-none-any.whl.metadata
  Obtaining dependency information for murmurhash<1.1.0,>=0.28.0 from https://files.pythonhosted.org/packages/71/46/af01a20ec368bd9cb49a1d2df15e3eca113bbf6952cc1f2a47f1c6801a7f/murmurhash-1.0.10-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for cymem<2

In [12]:
pip install transformers -q

Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install faiss-cpu -q

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install torch -q

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install streamlit -q

Note: you may need to restart the kernel to use updated packages.


In [7]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override = False)

False

In [13]:
import requests
import os
from bs4 import BeautifulSoup
import pandas as pd

# Function to fetch resumes from web links
def fetch_resumes(links):
    resumes = []
    for link in links:
        response = requests.get(link)
        soup = BeautifulSoup(response.content, 'html.parser')
        resumes.append(soup.get_text())
    return resumes

# Example web links
links = [
        'https://drive.google.com/file/d/14akVZY0QKo04F3-Ere_vDnIDznfWHaSd/view?usp=drive_link',
    'https://drive.google.com/file/d/1UMFACm2ivUE2MbPgJRfLs46yfOnuDkah/view?usp=drive_link',
    'https://drive.google.com/file/d/1j_RgoBeTV5AQwxkN7t-pdp87UiKoPv64/view?usp=drive_link'
    # Add more links as needed
]

# Fetch resumes
resumes = fetch_resumes(links)

# Save resumes to a file
with open('resumes.txt', 'w') as file:
    for resume in resumes:
        file.write(resume + '\n')

# Load resumes into a dataframe
df = pd.DataFrame(resumes, columns=['resume_text'])

# Preprocess resumes to extract metadata (this is a simplified example)
def extract_metadata(resume):
    # Extract work experience, education, etc.
    return {'work_experience': 'example', 'education': 'example'}

df['metadata'] = df['resume_text'].apply(extract_metadata)

# Save preprocessed resumes
df.to_csv('preprocessed_resumes.csv', index=False)


In [14]:
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Tokenize resumes
tokens = tokenizer(df['resume_text'].tolist(), padding=True, truncation=True, return_tensors='pt')

# Create embeddings
with torch.no_grad():
    embeddings = model(**tokens)

# Save embeddings
torch.save(embeddings, 'embeddings.pt')


In [15]:
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Tokenize resumes
tokens = tokenizer(df['resume_text'].tolist(), padding=True, truncation=True, return_tensors='pt')

# Create embeddings
with torch.no_grad():
    embeddings = model(**tokens)

# Save embeddings
torch.save(embeddings, 'embeddings.pt')


In [16]:
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import torch
import numpy as np
import pandas as pd

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Example resumes
resumes = [
    "Python developer with experience scaling AWS infrastructure",
    "Computer vision developer who has built models for object identification",
    "Python developer with FastAPI experience",
    # Add more resumes as needed
]

# Tokenize resumes
tokens = tokenizer(resumes, padding=True, truncation=True, return_tensors='pt')

# Create embeddings
with torch.no_grad():
    outputs = model(**tokens)
    resume_embeddings = outputs.last_hidden_state.mean(dim=1)  # Mean pooling

# Example query
query = "Python developer with experience scaling AWS infrastructure"
query_tokens = tokenizer(query, return_tensors='pt')

# Create query embedding
with torch.no_grad():
    query_output = model(**query_tokens)
    query_embedding = query_output.last_hidden_state.mean(dim=1)  # Mean pooling

# Compute cosine similarity
similarities = cosine_similarity(query_embedding.numpy(), resume_embeddings.numpy())
similar_indices = np.argsort(similarities, axis=1).flatten()

# Retrieve top results
top_resumes = np.array(resumes)[similar_indices[:10]]  # Adjust number of results as needed

# Display top results
print(top_resumes)


['Computer vision developer who has built models for object identification'
 'Python developer with FastAPI experience'
 'Python developer with experience scaling AWS infrastructure']


In [17]:
import streamlit as st

# Load preprocessed resumes and embeddings
df = pd.read_csv('preprocessed_resumes.csv')
embeddings = torch.load('embeddings.pt')

# Streamlit UI
st.title('Resume Semantic Search')
query = st.text_input('Enter your query:')
if query:
    query_tokens = tokenizer(query, return_tensors='pt')

    # Create query embedding
    with torch.no_grad():
        query_output = model(**query_tokens)
        query_embedding = query_output.last_hidden_state.mean(dim=1)  # Mean pooling

    # Compute cosine similarity
    similarities = cosine_similarity(query_embedding.numpy(), embeddings.numpy())
    similar_indices = np.argsort(similarities, axis=1).flatten()

    # Retrieve top results
    top_resumes = df.iloc[similar_indices[:10]]  # Adjust number of results as needed

    # Display top results
    st.write(top_resumes)

In [19]:
pip install sentence_transformers -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pinecone-client streamlit pandas sentence-transformers -q

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install pdfplumber -q

Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install python-docx -q

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip uninstall docx -q

^C
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import requests
import pandas as pd

def load_resumes_from_folder(folder_path):
    resumes = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.pdf') or filename.endswith('.docx'):
            filepath = os.path.join(folder_path, filename)
            resumes.append((filename, open(filepath, 'rb').read()))
    return resumes

def load_resumes_from_links(links):
    resumes = []
    for link in links:
        response = requests.get(link)
        if response.status_code == 200:
            resumes.append((link, response.content))
    return resumes

# Example usage
#folder_path = 'path/to/resume/folder'
#resumes = load_resumes_from_folder(folder_path)

# Or from links
links = [r'https://drive.google.com/drive/folders/1A0GPGrH7rLlAFNJ-RftYzLp28fJ0kKSU?usp=sharing']
resumes = load_resumes_from_links(links)


In [3]:
import pdfplumber
import docx
from docx import Document

def parse_resume(file, file_type):
    if file_type == 'pdf':
        with pdfplumber.open(file) as pdf:
            return ''.join([page.extract_text() for page in pdf.pages])
    elif file_type == 'docx':
        doc = Document(file)
        return '\n'.join([para.text for para in doc.paragraphs])

# Example usage
parsed_resumes = [(name, parse_resume(file, 'pdf' if name.endswith('.pdf') else 'docx')) for name, file in resumes]


AttributeError: 'bytes' object has no attribute 'seek'

In [15]:
pip install pandas faiss-cpu streamlit sentence-transformers requests pdfplumber tqdm -q


Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
import requests
import pandas as pd

def download_file_from_google_drive(file_id, destination):
    URL = "https://drive.google.com/drive/folders/1A0GPGrH7rLlAFNJ-RftYzLp28fJ0kKSU"
    session = requests.Session()
    response = session.get(URL, params={'id': file_id}, stream=True)
    token = get_confirm_token(response)
    
    if token:
        params = {'id': file_id, 'confirm': token}
        response = session.get(URL, params=params, stream=True)
    
    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value
    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768
    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk:
                f.write(chunk)

def load_resumes_from_links(links):
    resumes = []
    for link in links:
        file_id = link.split('/')[-2]
        destination = f"{file_id}.pdf"
        download_file_from_google_drive(file_id, destination)
        resumes.append(destination)
    return resumes


In [16]:
import pdfplumber
from sentence_transformers import SentenceTransformer

def extract_text_from_pdf(pdf_path):
    try:
        with pdfplumber.open(pdf_path) as pdf:
            text = ''
            for page in pdf.pages:
                text += page.extract_text() or ''
        return text
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
        return ""

def embed_resumes(resumes):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = []
    for resume in resumes:
        text = extract_text_from_pdf(resume)
        if text:  # Proceed only if text extraction was successful
            embedding = model.encode(text)
            embeddings.append(embedding)
    return embeddings


In [17]:
import faiss
import numpy as np

def build_faiss_index(embeddings):
    d = len(embeddings[0])
    index = faiss.IndexFlatL2(d)
    index.add(np.array(embeddings))
    return index


In [20]:
def search_resumes(query, index, resumes):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    query_embedding = model.encode(query)
    D, I = index.search(np.array([query_embedding]), k=10)
    return [resumes[i] for i in I[0]]


In [ ]:
def display_results(query, index, resumes):
    results = search_resumes(query, index, resumes)
    print(f"Query: {query}")
    for result in results:
        print(result)

# Example usage
main()
if __name__ == "__main__":
    links = ["https://drive.google.com/drive/folders/1A0GPGrH7rLlAFNJ-RftYzLp28fJ0kKSU"]
    resumes = load_resumes_from_links(links)
    embeddings = embed_resumes(resumes)
    if embeddings:  # Proceed only if embeddings were successfully created
        index = build_faiss_index(embeddings)
        
        # Display results for a sample query
        sample_query = "Python developer with experience in machine learning"
        display_results(sample_query, index, resumes)
    else:
        print("No valid resumes found or embeddings were not created.")


In [ ]:
import streamlit as st

def main():
    st.title("Resume Semantic Search")
    query = st.text_input("Enter your query:")
    if st.button("Search"):
        results = search_resumes(query, index, resumes)
        for result in results:
            st.write(result)